# Huấn luyện Phân loại Giới tính với Dataset từ Kaggle
Notebook này huấn luyện mô hình CNN để phân loại giới tính sử dụng dataset Gender Classification đã tải về local.

**Nguồn Dataset**: https://www.kaggle.com/datasets/cashutosh/gender-classification-dataset

In [ ]:
# Import các thư viện cần thiết
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Khám phá cấu trúc dataset
data_dir = './dataset/gender'
print("Cấu trúc dataset:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Chỉ hiển thị 3 file đầu tiên
        print(f"{subindent}{file}")
    if len(files) > 3:
        print(f"{subindent}... và {len(files) - 3} file khác")

In [ ]:
# Định nghĩa các lớp giới tính (dựa trên cấu trúc dataset thực tế)
# Dataset sẽ tự động phát hiện các lớp từ tên thư mục
gender_labels = ['Nam', 'Nữ']  # Nhãn chuẩn để hiển thị
num_classes = 2

print(f"Huấn luyện với {num_classes} lớp giới tính: {gender_labels}")
print("Dataset sẽ tự động phát hiện cấu trúc thư mục cho tên lớp")

In [ ]:
# Tiền xử lý và tăng cường dữ liệu (giảm augmentation để training nhanh hơn)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,  # Giảm từ 20 xuống 10
    width_shift_range=0.1,  # Giảm từ 0.2 xuống 0.1
    height_shift_range=0.1,  # Giảm từ 0.2 xuống 0.1
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator cho dữ liệu validation (chỉ rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Tải dữ liệu training từ thư mục Training
# Tăng batch_size và giảm kích thước ảnh để training nhanh hơn
train_generator = train_datagen.flow_from_directory(
    directory="./dataset/gender/Training",
    target_size=(64, 64),  # Giảm từ 150x150 xuống 64x64
    batch_size=64,  # Tăng từ 32 lên 64
    class_mode='categorical',
    shuffle=True
)

# Tải dữ liệu validation từ thư mục Validation
validation_generator = validation_datagen.flow_from_directory(
    directory="./dataset/gender/Validation",
    target_size=(64, 64),  # Giảm từ 150x150 xuống 64x64
    batch_size=64,  # Tăng từ 32 lên 64
    class_mode='categorical',
    shuffle=False
)

# In thông tin các lớp được phát hiện
print(f"Các lớp được phát hiện: {train_generator.class_indices}")
print(f"Số lượng mẫu training: {train_generator.n}")
print(f"Số lượng mẫu validation: {validation_generator.n}")

In [ ]:
# Xây dựng mô hình CNN cho phân loại giới tính (tối ưu hóa cho tốc độ)
model = Sequential()

# Các lớp Convolutional (giảm số lớp để training nhanh hơn)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten
model.add(Flatten())

# Các lớp fully-connected
model.add(Dense(64, activation='relu'))  # Giảm từ 128 xuống 64
model.add(Dropout(0.3))  # Tăng từ 0.25 lên 0.3

# Lớp output (2 lớp cho giới tính)
model.add(Dense(2, activation='softmax'))

# Hiển thị kiến trúc mô hình
model.summary()

In [ ]:
# Compile mô hình với optimizer Adam (thường nhanh hơn RMSprop)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks để tối ưu hóa training
early_stopping = EarlyStopping(
    patience=3, 
    monitor='val_loss', 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=0.0001
)

In [ ]:
# Huấn luyện mô hình (giảm epochs để training nhanh hơn)
print("Bắt đầu huấn luyện mô hình...")
history = model.fit(
    train_generator,
    steps_per_epoch=min(50, train_generator.n // train_generator.batch_size),  # Giới hạn steps
    epochs=10,  # Giảm từ 20 xuống 10
    validation_data=validation_generator,
    validation_steps=min(25, validation_generator.n // validation_generator.batch_size),  # Giới hạn validation steps
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
print("Hoàn thành huấn luyện!")

In [ ]:
# Đánh giá mô hình
score = model.evaluate(validation_generator, verbose=0)
print(f'Loss trên tập test: {score[0]:.4f}')
print(f'Độ chính xác trên tập test: {score[1]:.4f}')

In [ ]:
# Vẽ biểu đồ lịch sử huấn luyện
plt.figure(figsize=(12, 4))

# Vẽ biểu đồ độ chính xác
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Độ chính xác Training')
plt.plot(history.history['val_accuracy'], label='Độ chính xác Validation')
plt.title('Độ chính xác của Mô hình')
plt.ylabel('Độ chính xác')
plt.xlabel('Epoch')
plt.legend()

# Vẽ biểu đồ loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'r', label='Loss Training')
plt.plot(history.history['val_loss'], label='Loss Validation')
plt.title('Loss của Mô hình')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Lưu mô hình đã huấn luyện
model.save('Gender1.h5')  # Đổi tên từ Gender2 thành Gender1 để nhất quán
print("Mô hình đã được lưu với tên 'Gender1.h5'")

In [ ]:
# Kiểm tra mô hình với ảnh mẫu
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Tải mô hình đã lưu
model_CNN = load_model('Gender1.h5')  # Cập nhật tên file

# Định nghĩa nhãn dự đoán
predict_labels = ['Nam', 'Nữ']

# Hàm dự đoán giới tính từ đường dẫn ảnh
def predict_gender(image_path):
    try:
        # Tải và tiền xử lý ảnh
        img = load_img(image_path, target_size=(64, 64))  # Cập nhật kích thước
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title('Ảnh đầu vào')
        plt.axis('off')
        plt.show()
        
        # Chuyển đổi thành array và chuẩn hóa
        img_array = img_to_array(img)
        img_array = img_array.reshape(1, 64, 64, 3)  # Cập nhật kích thước
        img_array = img_array.astype('float32') / 255.0
        
        # Thực hiện dự đoán
        prediction = model_CNN.predict(img_array)
        predicted_class = np.argmax(prediction, axis=-1)[0]
        confidence = np.max(prediction)
        
        print(f"Giới tính dự đoán: {predict_labels[predicted_class]}")
        print(f"Độ tin cậy: {confidence:.2%}")
        
        return predict_labels[predicted_class], confidence
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh: {e}")
        return None, None

# Ví dụ sử dụng (bỏ comment và cung cấp đường dẫn ảnh hợp lệ)
# predict_gender('./dataset/gender/Training/female/sample_image.jpg')